In [22]:
import torch
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained('gpt2')

In [ ]:
import wandb

In [ ]:
run = wandb.init(project="Head Word Final 2")

In [ ]:
api = wandb.Api()

In [ ]:
runs = {}
for run in api.runs(
    path="ananthag/Head Word Final 2"
):
    runs[run.config['model_name']] = run.id

In [36]:
tok.convert_tokens_to_ids('bite')

37018

In [5]:
tok.decode(4532)

CodeGenTokenizerFast(name_or_path='microsoft/phi-2', vocab_size=50257, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50257: AddedToken("                               ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50258: AddedToken("                              ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50259: AddedToken("                             ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50260: AddedToken("                            ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50261: AddedToken("            

In [13]:
# tok.encode(' adjust', add_prefix_space=False)
type(tok)

transformers.models.codegen.tokenization_codegen_fast.CodeGenTokenizerFast

In [23]:
not tok.pad_token_id

True

In [14]:
inputs = ["Hey what's your name", "Welcome home", "text" * 20]
tok.pad_token_id = tok.eos_token_id
inputs_dict = tok.batch_encode_plus(inputs, padding=True, return_tensors='pt')
from transformers import AutoModelForCausalLM

m = AutoModelForCausalLM.from_pretrained('gpt2')
out = m(**inputs_dict)

In [20]:
tok.pad_token_id

50256

In [83]:
inputs = inputs_dict["input_ids"]

In [57]:
attn_mask = inputs_dict["attention_mask"]

last_non_masked_idx = torch.sum(attn_mask, dim=1) - 1

In [49]:
position_ids = torch.tensor(
            [list(range(inputs.shape[1])) for i in range(inputs.shape[0])]
        )

In [61]:
for i, position_ids_slice in enumerate(position_ids):
    position_ids_slice[last_non_masked_idx[i] :] = position_ids_slice[
        last_non_masked_idx[i]
    ]

In [50]:
torch.arange(inputs.shape[1]).repeat(inputs.shape[0], 1)

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19]])

In [64]:
inputs[attn_mask] = 2

In [66]:
attn_mask

tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [80]:

inputs.scatter_(0, 1 - attn_mask, last_non_masked_idx.unsqueeze(1))
inputs


RuntimeError: Expected index [3, 20] to be smaller than self [3, 20] apart from dimension 0 and to be smaller size than src [3, 1]

In [98]:
inputs = inputs * attn_mask + (~attn_mask.bool() * last_non_masked_idx.unsqueeze(1).expand(-1, inputs.shape[1]))

tensor([[False, False, False, False, False,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False]])

In [99]:
inputs

tensor([[10814,   644,   338,   534,  1438,     4,     4,     4,     4,     4,
             4,     4,     4,     4,     4,     4,     4,     4,     4,     4],
        [14618,  1363,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1],
        [ 5239,  5239,  5239,  5239,  5239,  5239,  5239,  5239,  5239,  5239,
          5239,  5239,  5239,  5239,  5239,  5239,  5239,  5239,  5239,  5239]])